In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D, Conv2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
num_classes = 100
nb_epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

#Pre-process the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
datagen.fit(x_train)

169017344/169001437 [==============================] - 11s 0us/step


In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

94781440/94765736 [==============================] - 4s 0us/step


In [ ]:
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#from keras.callbacks import ModelCheckpoint, LearningRateScheduler

#checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
#                               verbose=1, save_best_only=True)
t=time.time()

history = model.fit_generator(datagen.flow(x_train, y_train,batch_size=64),
                              steps_per_epoch=x_train.shape[0] // 64,epochs=5,
                              validation_data=(x_test, y_test))
print('Training time: %s' % (t - time.time()))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
781/781 [==============================] - 1012s 1s/step - loss: 2.0190 - accuracy: 0.4815 - val_loss: 1.0208 - val_accuracy: 0.7006
Epoch 2/5
781/781 [==============================] - 1003s 1s/step - loss: 1.1546 - accuracy: 0.6667 - val_loss: 0.8399 - val_accuracy: 0.7489
Epoch 3/5
781/781 [==============================] - 997s 1s/step - loss: 0.9794 - accuracy: 0.7104 - val_loss: 0.7948 - val_accuracy: 0.7589
Epoch 4/5
781/781 [==============================] - 997s 1s/step - loss: 0.8728 - accuracy: 0.7402 - val_loss: 0.7463 - val_accuracy: 0.7745
Epoch 5/5
781/781 [==============================] - 996s 1s/step - loss: 0.8025 - accuracy: 0.7590 - val_loss: 0.7662 - val_accuracy: 0.7751
Training time: -5079.005810022354
